# Setup Environment - Configuração e Instalação

Notebook para configurar o ambiente e instalar dependências necessárias para os testes de validação do pvlib-service.

Execute este notebook **PRIMEIRO** antes dos outros notebooks.

In [ ]:
# Instalação das dependências necessárias
!pip install pandas numpy requests matplotlib seaborn jupyter ipython
!pip install pvlib-python scipy scikit-learn ipywidgets

# Verificar instalação
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Todas as dependências foram instaladas com sucesso!")
print(f"📊 Pandas versão: {pd.__version__}")
print(f"🔢 NumPy versão: {np.__version__}")
print(f"🌐 Requests disponível")
print(f"📈 Matplotlib disponível")
print(f"🎨 Seaborn disponível")

## 📦 Instalação de Pacotes

In [ ]:
# Instalar dependências necessárias
import subprocess
import sys

def install_package(package):
    """Instalar pacote usando pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} instalado com sucesso")
    except subprocess.CalledProcessError as e:
        print(f"❌ Erro ao instalar {package}: {e}")

# Lista de pacotes necessários
required_packages = [
    'pvlib',
    'pandas',
    'numpy', 
    'matplotlib',
    'seaborn',
    'requests',
    'jupyter',
    'ipywidgets'
]

print("🚀 Instalando dependências...")
for package in required_packages:
    install_package(package)

print("\n✅ Todas as dependências foram instaladas!")

## 🔍 Verificação da Instalação

In [ ]:
# Verificar se todos os pacotes podem ser importados
import importlib

modules_to_test = {
    'pvlib': 'PVLIB para cálculos fotovoltaicos',
    'pandas': 'Manipulação de dados',
    'numpy': 'Computação numérica',
    'matplotlib': 'Gráficos e visualizações',
    'seaborn': 'Visualizações estatísticas',
    'requests': 'Requisições HTTP',
    'json': 'Manipulação JSON (built-in)',
    'datetime': 'Manipulação de datas (built-in)'
}

print("🔍 Verificando importações...\n")

all_good = True
for module, description in modules_to_test.items():
    try:
        imported_module = importlib.import_module(module)
        version = getattr(imported_module, '__version__', 'N/A')
        print(f"✅ {module:<12} v{version:<8} - {description}")
    except ImportError as e:
        print(f"❌ {module:<12} {'ERRO':<8} - {description} | {e}")
        all_good = False

if all_good:
    print("\n🎉 Todas as importações funcionaram corretamente!")
else:
    print("\n⚠️ Alguns módulos não puderam ser importados. Verifique a instalação.")

## 🧪 Teste Rápido do PVLIB

In [ ]:
# Teste básico do PVLIB
import pvlib
from pvlib import location, pvsystem
import pandas as pd
import numpy as np

print(f"🔬 Testando PVLIB v{pvlib.__version__}...\n")

try:
    # Criar uma localização (São Paulo)
    sao_paulo = location.Location(latitude=-23.5505, longitude=-46.6333, tz='America/Sao_Paulo')
    print(f"📍 Localização criada: {sao_paulo}")
    
    # Calcular posição solar para um dia
    times = pd.date_range('2024-06-21 06:00', '2024-06-21 18:00', freq='H', tz=sao_paulo.tz)
    solar_position = sao_paulo.get_solarposition(times)
    print(f"🌞 Posição solar calculada para {len(times)} horários")
    print(f"   Elevação máxima: {solar_position['elevation'].max():.1f}°")
    print(f"   Azimute ao meio-dia: {solar_position.loc[solar_position['elevation'].idxmax(), 'azimuth']:.1f}°")
    
    # Calcular irradiância clear-sky
    clearsky = sao_paulo.get_clearsky(times)
    print(f"☀️ Irradiância clear-sky calculada")
    print(f"   GHI máximo: {clearsky['ghi'].max():.0f} W/m²")
    print(f"   DNI máximo: {clearsky['dni'].max():.0f} W/m²")
    
    # Teste de sistema PV básico
    module_parameters = pvlib.pvsystem.retrieve_sam('SandiaMod')['Canadian_Solar_CS5P_220M___2009_']
    inverter_parameters = pvlib.pvsystem.retrieve_sam('cecinverter')['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
    
    print(f"🔋 Módulo teste carregado: {module_parameters['Material'][:30]}...")
    print(f"🔌 Inversor teste carregado: Potência {inverter_parameters['Paco']} W")
    
    print("\n✅ PVLIB está funcionando corretamente!")
    print("🎯 Pronto para executar os notebooks de análise.")
    
except Exception as e:
    print(f"❌ Erro no teste do PVLIB: {e}")
    print("⚠️ Verifique a instalação do PVLIB")

## 🌐 Teste de Conectividade (PVGIS)

In [ ]:
# Testar conectividade com PVGIS
import requests

print("🌐 Testando conectividade com PVGIS...")

try:
    # Fazer uma requisição simples para PVGIS
    url = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc"
    params = {
        'lat': -23.5505,
        'lon': -46.6333,
        'startyear': 2020,
        'endyear': 2020,
        'outputformat': 'json',
        'usehorizon': 1,
        'selectrad': 1,
        'angle': 0,
        'aspect': 0
    }
    
    print("🔄 Fazendo requisição teste para PVGIS...")
    response = requests.get(url, params=params, timeout=30)
    
    if response.status_code == 200:
        data = response.json()
        if 'outputs' in data and 'hourly' in data['outputs']:
            num_records = len(data['outputs']['hourly'])
            print(f"✅ PVGIS acessível! Recebidos {num_records:,} registros de teste")
            print(f"📊 Primeira entrada: {data['outputs']['hourly'][0]}")
        else:
            print("⚠️ PVGIS respondeu mas formato inesperado")
    else:
        print(f"❌ PVGIS retornou status {response.status_code}")
        
except requests.exceptions.Timeout:
    print("⏱️ Timeout na conexão com PVGIS (normal, servidor às vezes é lento)")
    print("💡 Os notebooks funcionarão, mas pode ser necessário aumentar timeout")
except requests.exceptions.RequestException as e:
    print(f"❌ Erro de conexão com PVGIS: {e}")
    print("🌐 Verifique sua conexão com a internet")
except Exception as e:
    print(f"❌ Erro inesperado: {e}")

## 📁 Criar Estrutura de Diretórios

In [ ]:
# Criar diretórios necessários
import os

directories = [
    '../data',
    '../outputs',
    '../reports'
]

print("📁 Criando estrutura de diretórios...")

for directory in directories:
    try:
        os.makedirs(directory, exist_ok=True)
        print(f"✅ {directory} criado/verificado")
    except Exception as e:
        print(f"❌ Erro ao criar {directory}: {e}")

# Criar arquivo README nos diretórios
readme_content = {
    '../data': "# Dados\nDiretório para dados processados dos notebooks\n- PVGIS data\n- Module calculations\n- Financial results\n",
    '../outputs': "# Outputs\nDiretório para saídas dos notebooks\n- Gráficos\n- Relatórios\n- Exportações\n",
    '../reports': "# Reports\nDiretório para relatórios finais\n- Análises completas\n- Comparações\n- Validações\n"
}

for directory, content in readme_content.items():
    try:
        with open(f"{directory}/README.md", "w", encoding='utf-8') as f:
            f.write(content)
        print(f"📝 README criado em {directory}")
    except Exception as e:
        print(f"❌ Erro ao criar README em {directory}: {e}")

print("\n✅ Estrutura de diretórios criada!")

## ⚙️ Configurações Globais

In [ ]:
# Configurações globais para os notebooks
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pandas as pd
import numpy as np

# Configurar warnings
warnings.filterwarnings('ignore')
print("⚠️ Warnings desabilitados")

# Configurar matplotlib
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 10
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
print("📊 Matplotlib configurado")

# Configurar seaborn
sns.set_palette("husl")
print("🎨 Seaborn configurado")

# Configurar pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)
print("🐼 Pandas configurado")

# Configurar numpy
np.random.seed(42)
print("🔢 NumPy configurado")

print("\n✅ Configurações globais aplicadas!")

## ✅ Verificação Final

In [ ]:
# Verificação final do ambiente
import os
import sys

print("🔍 VERIFICAÇÃO FINAL DO AMBIENTE")
print("=" * 40)

# Python version
print(f"🐍 Python: {sys.version}")

# Verificar se todos os módulos estão disponíveis
critical_modules = ['pvlib', 'pandas', 'numpy', 'matplotlib', 'requests']
missing_modules = []

for module in critical_modules:
    try:
        __import__(module)
        print(f"✅ {module}")
    except ImportError:
        print(f"❌ {module}")
        missing_modules.append(module)

# Verificar diretórios
required_dirs = ['../data', '../outputs', '../reports']
missing_dirs = []

for directory in required_dirs:
    if os.path.exists(directory):
        print(f"📁 {directory} ✅")
    else:
        print(f"📁 {directory} ❌")
        missing_dirs.append(directory)

# Verificar espaço em disco
import shutil
total, used, free = shutil.disk_usage(".")
free_gb = free // (2**30)
print(f"💾 Espaço livre: {free_gb} GB")

print("\n" + "=" * 40)

if not missing_modules and not missing_dirs:
    print("🎉 AMBIENTE PRONTO!")
    print("\n📋 PRÓXIMOS PASSOS:")
    print("1. Execute o notebook 01_pvgis_data_analysis.ipynb")
    print("2. Execute o notebook 02_module_calculations.ipynb")
    print("3. Execute o notebook 03_financial_analysis.ipynb")
    print("\n💡 DICAS:")
    print("• Modifique as variáveis de entrada em cada notebook")
    print("• Os dados são salvos automaticamente entre notebooks")
    print("• Consulte os arquivos README.md em cada diretório")
else:
    print("⚠️ PROBLEMAS ENCONTRADOS:")
    if missing_modules:
        print(f"   Módulos faltando: {missing_modules}")
    if missing_dirs:
        print(f"   Diretórios faltando: {missing_dirs}")
    print("\n🔧 Execute este notebook novamente para corrigir")

print(f"\n📊 Memória disponível: {free_gb} GB")
if free_gb < 1:
    print("⚠️ Pouco espaço em disco - considere liberar espaço")
else:
    print("✅ Espaço em disco suficiente")